# Realtor Scrapy

Scrapy the real state website and retrieve house listing of a target price and retrieve the info.

### Import Dependencies

In [1]:
import os
from bs4 import BeautifulSoup
import requests
from splinter import Browser
import pandas as pd
import time
import pprint

# Import DB user and password
from api_keys import positionstack_key
from api_keys import opencagedata_API

### Setup Splinter (For Mac)

In [22]:
# identify location of chromedriver and store it as a variable
driverPath = !which chromedriver

# Setup configuration variables to enable Splinter to interact with browser
executable_path = {'executable_path': driverPath[0]}
browser = Browser('chrome', **executable_path, headless=False)

### Scraping

In [23]:
# URL of page to be scraped
url_realtor = "https://www.realtor.com/realestateandhomes-search/Houston_TX/type-single-family-home/price-"
link_details = "https://www.realtor.com"
min_price = '250000'
max_price = '300000'
sort_by = '/sby-2' # Highest to lowest price
sort_by = '/sby-6' # Newest listings
page_number = 1

query_url = f"{url_realtor}{min_price}-{max_price}{sort_by}/pg-{page_number}"
print(query_url)

https://www.realtor.com/realestateandhomes-search/Houston_TX/type-single-family-home/price-250000-300000/sby-6/pg-1


### BeautifulSoup

In [24]:
# Scrap with BeautifulSoup. However it does 
# page = requests.get(query_url)
# soup = BeautifulSoup(page.content, 'html.parser')

### Splinter

In [25]:
# Use the browser to visit the url
browser.visit(query_url)

In [26]:
# Wait for x seconds for error purpouses
time.sleep(5)

In [27]:
# Return the rendered page by the browser
html_realtor = browser.html

In [28]:
# Use beatifulsoup to scrap the page rendered by the browser
soup = BeautifulSoup(html_realtor, 'html.parser')

In [29]:
# Search for the div where the title is located
results = soup.find_all('div', class_="card-box")
# print(results[1].prettify())
print(f"Total results: {len(results)}")
print('-----------------------------------')

news_title = results[1].text
# print(f"Title: {news_title}")
house_price = results[1].find('span', class_="sc-pZnSc jEfqsK")
# print(f"Price: {house_price.text}")
img_label = results[1].find('img', class_="fade top")
# print(f"Address: {img_label['alt']}")
link_page = results[1].find('a')['href']


print(f"Price: {house_price.text} | Address: {img_label['alt']}")
print(f"Link: {link_details}{link_page}")
print(f"Photo link: {img_label['srcset'].split(',')[1].split(' ')[1]}")

Total results: 48
-----------------------------------
Price: $263,400 | Address: 907 Willowick Bay Dr, Houston, TX 77090
Link: https://www.realtor.com/realestateandhomes-detail/907-Willowick-Bay-Dr_Houston_TX_77090_M95563-02852
Photo link: https://ap.rdcpix.com/5ce49a7700415a8f02f39e1335dd4739l-m2442378371od-w480_h360_x2.jpg


In [30]:
# Print results and save to a dictionary
n = 0
realstate_list = []
for result in results:
#     Clear the variables to not store repeated info
    house_price = ''
    address = ''
    link_page = ''
    photo_url = ''
                
    n = n + 1
    print('-----------------------------------')
    print('')
    print(f'Result: {n} of {len(results)}')
    if not result.find('div', class_="ads"):
#         print(f'Result: {n} of {len(results)}')
        price_div = result.find('div', class_="price")
        house_price = price_div.find('span').text.split('$')[-1]
        link_page = result.find('a')['href']
        img_label = result.find('img')
        address = img_label['alt']
        print(f"Price: ${house_price} | Address: {address}")
        try:
            price_reduced = result.find('span', class_="price-reduced-amount")
            price_now = price_reduced.text
            print(f"Price reduced: {price_now}")
        except:
            pass
        print(f"Link: {link_details}{link_page}")
#         photo_url = img_label['srcset'].split(',')[1].split(' ')[1]
        try:
            photo_url = img_label['srcset'].split(',')[1].split(' ')[1]
            print(f"Photo link: {photo_url}")
        except KeyError:
            print(f"Photo not available")
    
        # Save results to a dictionary
        realstate_list.append(
            {
                "Price": int(house_price.replace(',','')),
                "Address": address,
                "Link": str(link_details+link_page),
                "Photo link": photo_url
            }
        )

    
    else:
        print('Data not available')

    print('')
    
    

-----------------------------------

Result: 1 of 48
Price: $289,900 | Address: 2106 Sam Wilson St, Houston, TX 77020
Link: https://www.realtor.com/realestateandhomes-detail/Sam-Wilson_Houston_TX_77020_M80034-94461
Photo link: https://ap.rdcpix.com/93fe1181cc9f9158f8228fda18a4d809l-m1028545145od-w480_h360_x2.jpg

-----------------------------------

Result: 2 of 48
Price: $263,400 | Address: 907 Willowick Bay Dr, Houston, TX 77090
Link: https://www.realtor.com/realestateandhomes-detail/907-Willowick-Bay-Dr_Houston_TX_77090_M95563-02852
Photo link: https://ap.rdcpix.com/5ce49a7700415a8f02f39e1335dd4739l-m2442378371od-w480_h360_x2.jpg

-----------------------------------

Result: 3 of 48
Price: $265,000 | Address: 6602 Preston Trail Dr, Houston, TX 77069
Link: https://www.realtor.com/realestateandhomes-detail/6602-Preston-Trail-Dr_Houston_TX_77069_M86931-73332
Photo link: https://ap.rdcpix.com/d596621fe1117c2d539a69132be7c598l-m589595500od-w480_h360_x2.jpg

------------------------------

In [31]:
# When you’ve finished testing, close your browser using browser.quit:
browser.quit()

### Data Cleaning

In [32]:
# Save the data to a dataframe
listing_df = pd.DataFrame(realstate_list)
listing_df.head()

,Price,Address,Link,Photo link
0,289900,"2106 Sam Wilson St, Houston, TX 77020",https://www.realtor.com/realestateandhomes-det...,https://ap.rdcpix.com/93fe1181cc9f9158f8228fda...
1,263400,"907 Willowick Bay Dr, Houston, TX 77090",https://www.realtor.com/realestateandhomes-det...,https://ap.rdcpix.com/5ce49a7700415a8f02f39e13...
2,265000,"6602 Preston Trail Dr, Houston, TX 77069",https://www.realtor.com/realestateandhomes-det...,https://ap.rdcpix.com/d596621fe1117c2d539a6913...
3,299000,"5835 Duxbury St, Houston, TX 77035",https://www.realtor.com/realestateandhomes-det...,https://maps.googleapis.com/maps/api/streetvie...
4,252900,"9307 Windswept Grove Dr, Houston, TX 77083",https://www.realtor.com/realestateandhomes-det...,https://ap.rdcpix.com/9db15144e8578fd8ea3dfd55...


In [13]:
# Dataframe to dictionary
listing_df.to_dict(orient="records")

[{'Price': 300000,
  'Address': '3506 Chenevert St, Houston, TX 77004',
  'Link': 'https://www.realtor.com/realestateandhomes-detail/3506-Chenevert-St_Houston_TX_77004_M83831-74404',
  'Photo link': 'https://ap.rdcpix.com/c32a0999eb72fd3688f358dd5a9631e1l-m4050216900od-w480_h360_x2.jpg'},
 {'Price': 300000,
  'Address': '3923 Shadow Cove Dr, Houston, TX 77082',
  'Link': 'https://www.realtor.com/realestateandhomes-detail/3923-Shadow-Cove-Dr_Houston_TX_77082_M74659-12086',
  'Photo link': 'https://ap.rdcpix.com/8d85bba63c70d3a7ca27d84cfb40875al-m3947304570od-w480_h360_x2.jpg'},
 {'Price': 300000,
  'Address': '2620 Starboard Point Dr, Houston, TX 77054',
  'Link': 'https://www.realtor.com/realestateandhomes-detail/2620-Starboard-Point-Dr_Houston_TX_77054_M80817-72232',
  'Photo link': 'https://ap.rdcpix.com/49197c51ea457aa035238dbad29a1a23l-m1155526776od-w480_h360_x2.jpg'},
 {'Price': 300000,
  'Address': '10650 Alcott Dr, Houston, TX 77043',
  'Link': 'https://www.realtor.com/realestat

### Get Coordinates

In [90]:
# https://nominatim.org/
url_address = 'https://nominatim.openstreetmap.org/search?q='
address_search = '15747+Foxgate+Rd+Houston'
searchformat = '&format=json&polygon_geojson=1&addressdetails=1'
coordinate_search = f"{url_address}{address_search}{searchformat}"
print(coordinate_search)

#  Perform a request for data
response_coordinates = requests.get(coordinate_search).json()

https://nominatim.openstreetmap.org/search?q=15747+Foxgate+Rd+Houston&format=json&polygon_geojson=1&addressdetails=1


In [91]:
response_coordinates

[{'place_id': 291177600,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'osm_type': 'way',
  'osm_id': 15417204,
  'boundingbox': ['29.775993', '29.776093', '-95.635814', '-95.635714'],
  'lat': '29.776043',
  'lon': '-95.635764',
  'display_name': '15747, Foxgate Road, Addicks, Houston, Harris County, Texas, 77079, United States',
  'class': 'place',
  'type': 'house',
  'importance': 0.32100000000000006,
  'address': {'house_number': '15747',
   'road': 'Foxgate Road',
   'hamlet': 'Addicks',
   'city': 'Houston',
   'county': 'Harris County',
   'state': 'Texas',
   'postcode': '77079',
   'country': 'United States',
   'country_code': 'us'}}]

#### Complete Script To Get Coordinates (nominatim)

Note: during the project, it was observed that nominatin API was not able to handle all the addresses.

In [92]:
# https://nominatim.org/
url_address = 'https://nominatim.openstreetmap.org/search?q='
searchformat = '&format=json&polygon_geojson=1&addressdetails=1'
for index,row in listing_df.iterrows():
    print('-------------------------------------------------')
    # Format address to search for coordinates
    address = listing_df.iloc[index].Address.split(' ')
    x = "+".join(address)
    address_search = x.replace(',','')
    coordinate_search = f"{url_address}{address_search}{searchformat}"
    print(address_search)
    
    #  Perform a request for data
    response_coordinates = requests.get(coordinate_search).json()
    print(response_coordinates)

-------------------------------------------------
3506+Chenevert+St+Houston+TX+77004
[{'place_id': 291286915, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 138488478, 'boundingbox': ['29.735501795918', '29.735601795918', '-95.371899530612', '-95.371799530612'], 'lat': '29.73555179591837', 'lon': '-95.37184953061224', 'display_name': '3506, Chenevert Street, Houston, Harris County, Texas, 77004, United States', 'class': 'place', 'type': 'house', 'importance': 0.5309999999999999, 'address': {'house_number': '3506', 'road': 'Chenevert Street', 'city': 'Houston', 'county': 'Harris County', 'state': 'Texas', 'postcode': '77004', 'country': 'United States', 'country_code': 'us'}}]
-------------------------------------------------
3923+Shadow+Cove+Dr+Houston+TX+77082
[{'place_id': 291385965, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 15396639, 'boundingbox

[{'place_id': 291220665, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 15416739, 'boundingbox': ['29.841783444444', '29.841883444444', '-95.405213888889', '-95.405113888889'], 'lat': '29.841833444444443', 'lon': '-95.4051638888889', 'display_name': '410, Hohldale Street, Independence Heights, Houston, Harris County, Texas, 77091, United States', 'class': 'place', 'type': 'house', 'importance': 0.5309999999999999, 'address': {'house_number': '410', 'road': 'Hohldale Street', 'neighbourhood': 'Independence Heights', 'city': 'Houston', 'county': 'Harris County', 'state': 'Texas', 'postcode': '77091', 'country': 'United States', 'country_code': 'us'}}]
-------------------------------------------------
10819+Idlebrook+Dr+Houston+TX+77070
[]
-------------------------------------------------
3030+Triway+Ln+Houston+TX+77043
[{'place_id': 291393737, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyri

[]
-------------------------------------------------
26+N+Champion+Villa+Dr+Houston+TX+77069
[]
-------------------------------------------------
6814+Jensen+Dr+Houston+TX+77093
[{'place_id': 291321134, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 240336345, 'boundingbox': ['29.820979848485', '29.821079848485', '-95.342103666667', '-95.342003666667'], 'lat': '29.821029848484848', 'lon': '-95.34205366666666', 'display_name': '6814, Jensen Drive, Houston, Harris County, Texas, 77093, United States', 'class': 'place', 'type': 'house', 'importance': 0.5309999999999999, 'address': {'house_number': '6814', 'road': 'Jensen Drive', 'city': 'Houston', 'county': 'Harris County', 'state': 'Texas', 'postcode': '77093', 'country': 'United States', 'country_code': 'us'}}]
-------------------------------------------------
2649+Bellwick+Canyon+Ct+Pearland+TX+77089
[]
-------------------------------------------------
406+Oriole+St+Hou

In [107]:
def nominatim_coordinates(address):    
    # https://nominatim.org/
    url_address = 'https://nominatim.openstreetmap.org/search?q='
    searchformat = '&format=json&polygon_geojson=1&addressdetails=1'
    
    # Format address to search for coordinates
    address_string = address.split(' ')
    x = "+".join(address_string)
    address_search = x.replace(',','')
    coordinate_search = f"{url_address}{address_search}{searchformat}"
    
    #  Perform a request for data
    response_coordinates = requests.get(coordinate_search).json()
    
    # Handle empty results
    if not response_coordinates:
        valid_result = False
        print("No result found")
        return {"Valid": valid_result}
        
    else:
        valid_result = True
        print(response_coordinates)
    
        # return coordinates
        latitude = float(response_coordinates[0]['lat'])
        longitude = float(response_coordinates[0]['lon'])

        map_url = f"https://www.openstreetmap.org/?mlat={latitude}&mlon={longitude}#map=15/{latitude}/{longitude}"
        print(f"Map: {map_url}")

        return {"Valid": valid_result,"latitude": latitude, "longitude": longitude, "map_url": map_url}

In [96]:
# Example result found
nominatim_coordinates('9610+Chatfield+St+Houston+TX+77025')

[{'place_id': 291295118, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 361742396, 'boundingbox': ['29.681077494432', '29.681177494432', '-95.427341284522', '-95.427241284522'], 'lat': '29.68112749443186', 'lon': '-95.42729128452169', 'display_name': '9610, Chatfield Street, Westridge, Houston, Harris County, Texas, 77025, United States', 'class': 'place', 'type': 'house', 'importance': 0.5309999999999999, 'address': {'house_number': '9610', 'road': 'Chatfield Street', 'residential': 'Westridge', 'city': 'Houston', 'county': 'Harris County', 'state': 'Texas', 'postcode': '77025', 'country': 'United States', 'country_code': 'us'}}]
Map: https://www.openstreetmap.org/?mlat=29.68&mlon=-95.43#map=15/29.68/-95.43


{'Valid': True,
 'latitude': 29.68,
 'longitude': -95.43,
 'map': 'https://www.openstreetmap.org/?mlat=29.68&mlon=-95.43#map=15/29.68/-95.43'}

In [97]:
# Example of no result found
nominatim_coordinates('650+Westcross+St+Unit+38+Houston+TX+77018')

No result found


{'Valid': False}

In [98]:
for index,row in listing_df.iterrows():
    print('-------------------------------------------------')
    # Format address to search for coordinates
    address = listing_df.iloc[index].Address
    print(address)
    nominatim_coordinates(address)

-------------------------------------------------
3506 Chenevert St, Houston, TX 77004
[{'place_id': 291286915, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 138488478, 'boundingbox': ['29.735501795918', '29.735601795918', '-95.371899530612', '-95.371799530612'], 'lat': '29.73555179591837', 'lon': '-95.37184953061224', 'display_name': '3506, Chenevert Street, Houston, Harris County, Texas, 77004, United States', 'class': 'place', 'type': 'house', 'importance': 0.5309999999999999, 'address': {'house_number': '3506', 'road': 'Chenevert Street', 'city': 'Houston', 'county': 'Harris County', 'state': 'Texas', 'postcode': '77004', 'country': 'United States', 'country_code': 'us'}}]
Map: https://www.openstreetmap.org/?mlat=29.74&mlon=-95.37#map=15/29.74/-95.37
-------------------------------------------------
3923 Shadow Cove Dr, Houston, TX 77082
[{'place_id': 291385965, 'licence': 'Data © OpenStreetMap contributors, ODbL 1

[{'place_id': 291197680, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 15423824, 'boundingbox': ['29.779674591837', '29.779774591837', '-95.343884795918', '-95.343784795918'], 'lat': '29.779724591836736', 'lon': '-95.34383479591837', 'display_name': '2006, Stevens Street, Houston, Harris County, Texas, 77026, United States', 'class': 'place', 'type': 'house', 'importance': 0.5309999999999999, 'address': {'house_number': '2006', 'road': 'Stevens Street', 'city': 'Houston', 'county': 'Harris County', 'state': 'Texas', 'postcode': '77026', 'country': 'United States', 'country_code': 'us'}}]
Map: https://www.openstreetmap.org/?mlat=29.78&mlon=-95.34#map=15/29.78/-95.34
-------------------------------------------------
410 Hohldale St, Houston, TX 77091
[{'place_id': 291220665, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 15416739, 'boundingbox': ['29.8417

No result found
-------------------------------------------------
12315 Macbrice Ln, Humble, TX 77346
No result found
-------------------------------------------------
1421 Bringhurst St, Houston, TX 77020
[{'place_id': 291291346, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 15368257, 'boundingbox': ['29.774352928571', '29.774452928571', '-95.334038', '-95.333938'], 'lat': '29.774402928571426', 'lon': '-95.33398799999999', 'display_name': '1421, Bringhurst Street, Houston, Harris County, Texas, 77020, United States', 'class': 'place', 'type': 'house', 'importance': 0.5309999999999999, 'address': {'house_number': '1421', 'road': 'Bringhurst Street', 'city': 'Houston', 'county': 'Harris County', 'state': 'Texas', 'postcode': '77020', 'country': 'United States', 'country_code': 'us'}}]
Map: https://www.openstreetmap.org/?mlat=29.77&mlon=-95.33#map=15/29.77/-95.33
-------------------------------------------------
5503 Che

No result found
-------------------------------------------------
643 Press St, Houston, TX 77020
[{'place_id': 91845593, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 15396739, 'boundingbox': ['29.772689', '29.774382', '-95.328856', '-95.328816'], 'lat': '29.774382', 'lon': '-95.328856', 'display_name': 'Press Street, Houston, Harris County, Texas, 77020, United States', 'class': 'highway', 'type': 'residential', 'importance': 0.5199999999999999, 'address': {'road': 'Press Street', 'city': 'Houston', 'county': 'Harris County', 'state': 'Texas', 'postcode': '77020', 'country': 'United States', 'country_code': 'us'}, 'geojson': {'type': 'LineString', 'coordinates': [[-95.328816, 29.772689], [-95.328856, 29.774382]]}}]
Map: https://www.openstreetmap.org/?mlat=29.77&mlon=-95.33#map=15/29.77/-95.33
-------------------------------------------------
622 Janisch Rd Unit L, Houston, TX 77018
No result found
-------------------

#### Complete Script To Get Coordinates (positionstack)

https://positionstack.com/quickstart

In [85]:
# Forward Geocoding API Endpoint
url_positiontrack = f"http://api.positionstack.com/v1/forward?access_key={positionstack_key}&query="
address = "650 Westcross St Unit 38, Houston, TX 77018"
address_lookup = address.replace(" ", "%20")
url_coordinates = f"{url_positiontrack}{address_lookup}&limit=1"

print(f"Query URL:")
print(url_coordinates)
print('')

Query URL:
http://api.positionstack.com/v1/forward?access_key=334c62c7568d1f1becd64b7dca301c0c&query=650%20Westcross%20St%20Unit%2038,%20Houston,%20TX%2077018&limit=1



In [86]:
#  Perform a request for data
response_coordinates = requests.get(url_coordinates).json()
print(response_coordinates)

{'data': [{'latitude': 29.829312, 'longitude': -95.409167, 'type': 'address', 'name': '650 Westcross St', 'number': '650', 'postal_code': '77018', 'street': 'Westcross St', 'confidence': 1, 'region': 'Texas', 'region_code': 'TX', 'county': None, 'locality': None, 'administrative_area': None, 'neighbourhood': 'Independence Heights', 'country': 'United States', 'country_code': 'USA', 'continent': 'North America', 'label': '650 Westcross St, TX, USA'}]}


In [87]:
len(response_coordinates['data'][0])

18

In [84]:
# Handle empty results
if len(response_coordinates['data'][0]) == 0:
    valid_result = False
    print("No result found")


else:
    valid_result = True
    print(response_coordinates)

    # return coordinates
    latitude = response_coordinates['data'][0]['latitude']
    longitude = response_coordinates['data'][0]['longitude']

    map_url = f"https://www.openstreetmap.org/?mlat={latitude}&mlon={longitude}#map=15/{latitude}/{longitude}"
    print(f"Map: {map_url}")

{'data': [{'latitude': 29.829312, 'longitude': -95.409167, 'type': 'address', 'name': '650 Westcross St', 'number': '650', 'postal_code': '77018', 'street': 'Westcross St', 'confidence': 1, 'region': 'Texas', 'region_code': 'TX', 'county': None, 'locality': None, 'administrative_area': None, 'neighbourhood': 'Independence Heights', 'country': 'United States', 'country_code': 'USA', 'continent': 'North America', 'label': '650 Westcross St, TX, USA'}]}
Map: https://www.openstreetmap.org/?mlat=29.829312&mlon=-95.409167#map=15/29.829312/-95.409167


In [106]:
def positiontrack_coordinates(address):
    # Forward Geocoding API Endpoint
    url_positiontrack = f"http://api.positionstack.com/v1/forward?access_key={positionstack_key}&query="
    
    address_lookup = address.replace(" ", "%20")
    url_coordinates = f"{url_positiontrack}{address_lookup}&limit=1"

    print(f"Query URL:")
    print(url_coordinates)
    print('')

    #  Perform a request for data
    response_coordinates = requests.get(url_coordinates).json()
    
    
    # Handle empty results
    if len(response_coordinates['data'][0]) == 0:
        valid_result = False
        print("No result found")
        return {"Valid": valid_result}
        
    else:
        valid_result = True
        print(response_coordinates)
    
        # return coordinates
        latitude = response_coordinates['data'][0]['latitude']
        longitude = response_coordinates['data'][0]['longitude']
        
        map_url = f"https://www.openstreetmap.org/?mlat={latitude}&mlon={longitude}#map=15/{latitude}/{longitude}"
        print(f"Map: {map_url}")

        return {"Valid": valid_result,"latitude": latitude, "longitude": longitude, "map_url": map_url}


In [66]:
# Example of query for coordinates
address = "650 Westcross St Unit 38, Houston, TX 77018"
positiontrack_coordinates(address)

Query URL:
http://api.positionstack.com/v1/forward?access_key=334c62c7568d1f1becd64b7dca301c0c&query=650%20Westcross%20St%20Unit%2038,%20Houston,%20TX%2077018& limit=1

{'data': [{'latitude': 29.829312, 'longitude': -95.409167, 'type': 'address', 'name': '650 Westcross St', 'number': '650', 'postal_code': '77018', 'street': 'Westcross St', 'confidence': 1, 'region': 'Texas', 'region_code': 'TX', 'county': None, 'locality': None, 'administrative_area': None, 'neighbourhood': 'Independence Heights', 'country': 'United States', 'country_code': 'USA', 'continent': 'North America', 'label': '650 Westcross St, TX, USA'}]}
Map: https://www.openstreetmap.org/?mlat=29.829312&mlon=-95.409167#map=15/29.829312/-95.409167


{'Valid': True,
 'latitude': 29.829312,
 'longitude': -95.409167,
 'map': 'https://www.openstreetmap.org/?mlat=29.829312&mlon=-95.409167#map=15/29.829312/-95.409167'}

In [61]:
# Example of query for coordinates
address = "17511 Lonesome Dove Trl, Houston, TX 77095"
positiontrack_coordinates(address)

Query URL:
http://api.positionstack.com/v1/forward?access_key=334c62c7568d1f1becd64b7dca301c0c&query=17511%20Lonesome%20Dove%20Trl,%20Houston,%20TX%2077095& limit=1

{'data': [{'latitude': 29.934773, 'longitude': -95.678385, 'type': 'address', 'name': '17511 Lonesome Dove Trl', 'number': '17511', 'postal_code': '77095', 'street': 'Lonesome Dove Trl', 'confidence': 1, 'region': 'Texas', 'region_code': 'TX', 'county': None, 'locality': None, 'administrative_area': None, 'neighbourhood': None, 'country': 'United States', 'country_code': 'USA', 'continent': 'North America', 'label': '17511 Lonesome Dove Trl, TX, USA'}]}
Map: https://www.openstreetmap.org/?mlat=29.934773&mlon=-95.678385#map=15/29.934773/-95.678385


{'Valid': True,
 'latitude': 29.934773,
 'longitude': -95.678385,
 'map': 'https://www.openstreetmap.org/?mlat=29.934773&mlon=-95.678385#map=15/29.934773/-95.678385'}

In [63]:
# Example of query for coordinates
address = "10418 Springland Ct, Houston,TX 77065"
positiontrack_coordinates(address)


Query URL:
http://api.positionstack.com/v1/forward?access_key=334c62c7568d1f1becd64b7dca301c0c&query=10418%20Springland%20Ct,%20Houston,TX%2077065& limit=1

{'data': [{'latitude': 29.924385, 'longitude': -95.591604, 'type': 'address', 'name': '10418 Springland Ct', 'number': '10418', 'postal_code': '77065', 'street': 'Springland Ct', 'confidence': 1, 'region': 'Texas', 'region_code': 'TX', 'county': None, 'locality': None, 'administrative_area': None, 'neighbourhood': 'Willowbrook', 'country': 'United States', 'country_code': 'USA', 'continent': 'North America', 'label': '10418 Springland Ct, TX, USA'}]}
Map: https://www.openstreetmap.org/?mlat=29.924385&mlon=-95.591604#map=15/29.924385/-95.591604


{'Valid': True,
 'latitude': 29.924385,
 'longitude': -95.591604,
 'map': 'https://www.openstreetmap.org/?mlat=29.924385&mlon=-95.591604#map=15/29.924385/-95.591604'}

In [69]:
address = "9610 Chatfield St, Houston, TX 77025"
positiontrack_coordinates(address)

Query URL:
http://api.positionstack.com/v1/forward?access_key=334c62c7568d1f1becd64b7dca301c0c&query=9610%20Chatfield%20St,%20Houston,%20TX%2077025& limit=1

{'data': [{'latitude': 29.68103, 'longitude': -95.427816, 'type': 'address', 'name': '9610 Chatfield St', 'number': '9610', 'postal_code': '77025', 'street': 'Chatfield St', 'confidence': 1, 'region': 'Texas', 'region_code': 'TX', 'county': None, 'locality': None, 'administrative_area': None, 'neighbourhood': 'Braeswood Place', 'country': 'United States', 'country_code': 'USA', 'continent': 'North America', 'label': '9610 Chatfield St, TX, USA'}]}
Map: https://www.openstreetmap.org/?mlat=29.68103&mlon=-95.427816#map=15/29.68103/-95.427816


{'Valid': True,
 'latitude': 29.68103,
 'longitude': -95.427816,
 'map': 'https://www.openstreetmap.org/?mlat=29.68103&mlon=-95.427816#map=15/29.68103/-95.427816'}

In [89]:
for index,row in listing_df.iterrows():
    print('-------------------------------------------------')
    # Format address to search for coordinates
    address = listing_df.iloc[index].Address
    print(address)
    positiontrack_coordinates(address)
#     time.sleep(3)

-------------------------------------------------
3506 Chenevert St, Houston, TX 77004
Query URL:
http://api.positionstack.com/v1/forward?access_key=334c62c7568d1f1becd64b7dca301c0c&query=3506%20Chenevert%20St,%20Houston,%20TX%2077004& limit=1

{'data': [{'latitude': 29.735441, 'longitude': -95.372164, 'type': 'address', 'name': '3506 Chenevert St', 'number': '3506', 'postal_code': '77004', 'street': 'Chenevert St', 'confidence': 1, 'region': 'Texas', 'region_code': 'TX', 'county': None, 'locality': None, 'administrative_area': None, 'neighbourhood': 'Midtown', 'country': 'United States', 'country_code': 'USA', 'continent': 'North America', 'label': '3506 Chenevert St, TX, USA'}]}
Map: https://www.openstreetmap.org/?mlat=29.735441&mlon=-95.372164#map=15/29.735441/-95.372164
-------------------------------------------------
3923 Shadow Cove Dr, Houston, TX 77082
Query URL:
http://api.positionstack.com/v1/forward?access_key=334c62c7568d1f1becd64b7dca301c0c&query=3923%20Shadow%20Cove%20Dr

{'data': [{'latitude': 29.924385, 'longitude': -95.591604, 'type': 'address', 'name': '10418 Springland Ct', 'number': '10418', 'postal_code': '77065', 'street': 'Springland Ct', 'confidence': 1, 'region': 'Texas', 'region_code': 'TX', 'county': None, 'locality': None, 'administrative_area': None, 'neighbourhood': 'Willowbrook', 'country': 'United States', 'country_code': 'USA', 'continent': 'North America', 'label': '10418 Springland Ct, TX, USA'}]}
Map: https://www.openstreetmap.org/?mlat=29.924385&mlon=-95.591604#map=15/29.924385/-95.591604
-------------------------------------------------
702 Tabor St, Houston, TX 77009
Query URL:
http://api.positionstack.com/v1/forward?access_key=334c62c7568d1f1becd64b7dca301c0c&query=702%20Tabor%20St,%20Houston,%20TX%2077009& limit=1

{'data': [{'latitude': 29.797364, 'longitude': -95.377012, 'type': 'address', 'name': '702 Tabor St', 'number': '702', 'postal_code': '77009', 'street': 'Tabor St', 'confidence': 1, 'region': 'Texas', 'region_code':

{'data': [{'latitude': 29.675144, 'longitude': -95.425053, 'type': 'address', 'name': '3006 Orchard Dr', 'number': '3006', 'postal_code': '77054', 'street': 'Orchard Dr', 'confidence': 1, 'region': 'Texas', 'region_code': 'TX', 'county': None, 'locality': None, 'administrative_area': None, 'neighbourhood': 'Willow Meadows', 'country': 'United States', 'country_code': 'USA', 'continent': 'North America', 'label': '3006 Orchard Dr, TX, USA'}]}
Map: https://www.openstreetmap.org/?mlat=29.675144&mlon=-95.425053#map=15/29.675144/-95.425053
-------------------------------------------------
2902 Tulip River Ct, Houston, TX 77345
Query URL:
http://api.positionstack.com/v1/forward?access_key=334c62c7568d1f1becd64b7dca301c0c&query=2902%20Tulip%20River%20Ct,%20Houston,%20TX%2077345& limit=1

No result found
-------------------------------------------------
1607 Blossom Creek Ct, Kingwood, TX 77339
Query URL:
http://api.positionstack.com/v1/forward?access_key=334c62c7568d1f1becd64b7dca301c0c&query

{'data': [{'latitude': 29.822676, 'longitude': -95.400483, 'type': 'address', 'name': '209 Cockerel St', 'number': '209', 'postal_code': '77018', 'street': 'Cockerel St', 'confidence': 1, 'region': 'Texas', 'region_code': 'TX', 'county': None, 'locality': None, 'administrative_area': None, 'neighbourhood': 'Independence Heights', 'country': 'United States', 'country_code': 'USA', 'continent': 'North America', 'label': '209 Cockerel St, TX, USA'}]}
Map: https://www.openstreetmap.org/?mlat=29.822676&mlon=-95.400483#map=15/29.822676/-95.400483
-------------------------------------------------
3123 Briarwood Point Place, Houston, TX 77063
Query URL:
http://api.positionstack.com/v1/forward?access_key=334c62c7568d1f1becd64b7dca301c0c&query=3123%20Briarwood%20Point%20Place,%20Houston,%20TX%2077063& limit=1

{'data': [{'latitude': 29.731016, 'longitude': -95.51441, 'type': 'address', 'name': '3123 Briarwood Point Pl', 'number': '3123', 'postal_code': '77063', 'street': 'Briarwood Point Pl', 'c

### Handle no result on nominatin and use position track

In [109]:
for index,row in listing_df.iterrows():
    print('-------------------------------------------------')
    # Format address to search for coordinates
    address = listing_df.iloc[index].Address
    print(address)
    
    # Try retrieve coordinates with nominatin
    result_nominatin = nominatim_coordinates(address)
    
    # if result not valid use position track
    if not result_nominatin["Valid"]:
        print('Use position track')
        result_positiontrack = positiontrack_coordinates(address)
        # If neither one bring the valid result
        if not result_positiontrack["Valid"]:
            print("No result found")
        else:
            valid_result = True
            result = result_positiontrack
            latitude = result['latitude']
            longitude = result['longitude']
            map_url = result['map_url']
            print(f"latitude: {latitude} \n longitude: {longitude} \n map_url: {map_url}")
    else:
        valid_result = True
        result = result_nominatin
        latitude = result['latitude']
        longitude = result['longitude']
        map_url = result['map_url']
        print(f"latitude: {latitude} \n longitude: {longitude} \n map_url: {map_url}")

    

-------------------------------------------------
3506 Chenevert St, Houston, TX 77004
[{'place_id': 291286915, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 138488478, 'boundingbox': ['29.735501795918', '29.735601795918', '-95.371899530612', '-95.371799530612'], 'lat': '29.73555179591837', 'lon': '-95.37184953061224', 'display_name': '3506, Chenevert Street, Houston, Harris County, Texas, 77004, United States', 'class': 'place', 'type': 'house', 'importance': 0.5309999999999999, 'address': {'house_number': '3506', 'road': 'Chenevert Street', 'city': 'Houston', 'county': 'Harris County', 'state': 'Texas', 'postcode': '77004', 'country': 'United States', 'country_code': 'us'}}]
Map: https://www.openstreetmap.org/?mlat=30&mlon=-95#map=15/30/-95
latitude: 30 
 longitude: -95 
 map_url: https://www.openstreetmap.org/?mlat=30&mlon=-95#map=15/30/-95
-------------------------------------------------
3923 Shadow Cove Dr, Houst

[{'place_id': 291343415, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 15415154, 'boundingbox': ['29.858034612245', '29.858134612245', '-95.423504020408', '-95.423404020408'], 'lat': '29.858084612244898', 'lon': '-95.42345402040816', 'display_name': '6510, Cohn Street, Highland Heights, Houston, Harris County, Texas, 77091, United States', 'class': 'place', 'type': 'house', 'importance': 0.5309999999999999, 'address': {'house_number': '6510', 'road': 'Cohn Street', 'neighbourhood': 'Highland Heights', 'city': 'Houston', 'county': 'Harris County', 'state': 'Texas', 'postcode': '77091', 'country': 'United States', 'country_code': 'us'}}]
Map: https://www.openstreetmap.org/?mlat=30&mlon=-95#map=15/30/-95
latitude: 30 
 longitude: -95 
 map_url: https://www.openstreetmap.org/?mlat=30&mlon=-95#map=15/30/-95
-------------------------------------------------
5318 S Oak Cove Dr, Houston, TX 77091
[{'place_id': 291193655, 'lice

[{'place_id': 291301532, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 15451741, 'boundingbox': ['29.867347244898', '29.867447244898', '-95.347155285714', '-95.347055285714'], 'lat': '29.86739724489796', 'lon': '-95.34710528571428', 'display_name': '10604, Park Lane, Houston, Harris County, Texas, 77093, United States', 'class': 'place', 'type': 'house', 'importance': 0.431, 'address': {'house_number': '10604', 'road': 'Park Lane', 'city': 'Houston', 'county': 'Harris County', 'state': 'Texas', 'postcode': '77093', 'country': 'United States', 'country_code': 'us'}}]
Map: https://www.openstreetmap.org/?mlat=30&mlon=-95#map=15/30/-95
latitude: 30 
 longitude: -95 
 map_url: https://www.openstreetmap.org/?mlat=30&mlon=-95#map=15/30/-95
-------------------------------------------------
4711 Hershe St, Houston, TX 77020
[{'place_id': 291291728, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyrig

No result found
Use position track
Query URL:
http://api.positionstack.com/v1/forward?access_key=334c62c7568d1f1becd64b7dca301c0c&query=12315%20Macbrice%20Ln,%20Humble,%20TX%2077346&limit=1

{'data': [{'latitude': 29.97205, 'longitude': -95.194155, 'type': 'address', 'name': '12315 Denali Ln', 'number': '12315', 'postal_code': '77346', 'street': 'Denali Ln', 'confidence': 1, 'region': 'Texas', 'region_code': 'TX', 'county': None, 'locality': None, 'administrative_area': None, 'neighbourhood': None, 'country': 'United States', 'country_code': 'USA', 'continent': 'North America', 'label': '12315 Denali Ln, TX, USA'}]}
Map: https://www.openstreetmap.org/?mlat=29.97205&mlon=-95.194155#map=15/29.97205/-95.194155
latitude: 29.97205 
 longitude: -95.194155 
 map_url: https://www.openstreetmap.org/?mlat=29.97205&mlon=-95.194155#map=15/29.97205/-95.194155
-------------------------------------------------
1421 Bringhurst St, Houston, TX 77020
[{'place_id': 291291346, 'licence': 'Data © OpenStree

No result found
Use position track
Query URL:
http://api.positionstack.com/v1/forward?access_key=334c62c7568d1f1becd64b7dca301c0c&query=3123%20Briarwood%20Point%20Place,%20Houston,%20TX%2077063&limit=1

No result found
No result found
-------------------------------------------------
4712 Libbey Ln, Houston, TX 77092
[{'place_id': 89839708, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 15357936, 'boundingbox': ['29.830832', '29.830987', '-95.4727524', '-95.462298'], 'lat': '29.830835', 'lon': '-95.466487', 'display_name': 'Libbey Lane, Oak Forest, Rosslyn, Houston, Harris County, Texas, 77092, United States', 'class': 'highway', 'type': 'residential', 'importance': 0.42000000000000004, 'address': {'road': 'Libbey Lane', 'neighbourhood': 'Oak Forest', 'hamlet': 'Rosslyn', 'city': 'Houston', 'county': 'Harris County', 'state': 'Texas', 'postcode': '77092', 'country': 'United States', 'country_code': 'us'}, 'geojson': {'t

In [ ]:
def address_coordinates(address):
    # Try retrieve coordinates with nominatin
    result_nominatin = nominatim_coordinates(address)
    
    # if result not valid use position track
    if not result_nominatin["Valid"]:
        print('Use position track')
        result_positiontrack = positiontrack_coordinates(address)
        # If neither one bring the valid result
        if not result_positiontrack["Valid"]:
            valid_result = False
            print("No result found")
            return {"Valid": valid_result}
        else:
            result = result_positiontrack
            latitude = result['latitude']
            longitude = result['longitude']
            map_url = result['map_url']
            return {"Valid": valid_result,"latitude": latitude, "longitude": longitude, "map_url": map_url}
    else:
        result = result_nominatin
        latitude = result['latitude']
        longitude = result['longitude']
        map_url = result['map_url']
        return {"Valid": valid_result,"latitude": latitude, "longitude": longitude, "map_url": map_url}

### Verify Coordinates

In [2]:
"https://nominatim.openstreetmap.org/search?q=1717+Allendale+Bluff+Ln+Pearland+TX+77089&format=json&polygon_geojson=1&addressdetails=1"

'https://nominatim.openstreetmap.org/search?q=1717+Allendale+Bluff+Ln+Pearland+TX+77089&format=json&polygon_geojson=1&addressdetails=1'

In [4]:
"http://api.positionstack.com/v1/forward?access_key=334c62c7568d1f1becd64b7dca301c0c&query=1717%20Allendale%20Bluff%20Ln%20Pearland%20TX%2077089&region=Houston"

'http://api.positionstack.com/v1/forward?access_key=334c62c7568d1f1becd64b7dca301c0c&query=1717%20Allendale%20Bluff%20Ln%20Pearland%20TX%2077089&region=Houston'

In [2]:
# Houston City Hall Coordinates
CityHallLat = 29.760376354375307
CityHallLon = -95.3702170895345

In [3]:
proximity=CityHallLat,CityHallLon

# opencagedata Forward Coordinates

In [12]:
url_opencagedata = 'https://api.opencagedata.com/geocode/v1/json?q='
api_KEY = f"&key={opencagedata_API}"

# Houston City Hall Coordinates
HoustonCityHallCoordinates = "29.760376354375307,-95.3702170895345"
proximity=f"{HoustonCityHallCoordinates}"

PLACENAME = "1717%20Allendale%20Bluff%20Ln%20Pearland%20TX%2077089"
bounds='-95.91830,29.35389,-94.80848,30.26282'  # Bounds for Houston (https://opencagedata.com/bounds-finder)
query_url = f"{url_opencagedata}{PLACENAME}{api_KEY}&bounds={bounds}&proximity={proximity}"
print(query_url)

https://api.opencagedata.com/geocode/v1/json?q=1717%20Allendale%20Bluff%20Ln%20Pearland%20TX%2077089&key=07e739bbcadd4cc88a3c2e307e0ad36f&bounds=-95.91830,29.35389,-94.80848,30.26282&proximity=29.760376354375307,-95.3702170895345


In [13]:
# url_opencagedata
response_coordinates = requests.get(query_url).json()

In [40]:
response_coordinates

{'documentation': 'https://opencagedata.com/api',
 'licenses': [{'name': 'see attribution guide',
   'url': 'https://opencagedata.com/credits'}],
 'rate': {'limit': 2500, 'remaining': 2496, 'reset': 1609286400},
 'results': [{'annotations': {'DMS': {'lat': "29° 40' 59.26800'' N",
     'lng': "95° 12' 59.43960'' W"},
    'FIPS': {'state': '48'},
    'MGRS': '15RTN8551985729',
    'Maidenhead': 'EL29jq43aw',
    'Mercator': {'x': -10599453.52, 'y': 3441728.248},
    'OSM': {'note_url': 'https://www.openstreetmap.org/note/new#map=16/29.68313/-95.21651&layers=N',
     'url': 'https://www.openstreetmap.org/?mlat=29.68313&mlon=-95.21651#map=16/29.68313/-95.21651'},
    'UN_M49': {'regions': {'AMERICAS': '019',
      'NORTHERN_AMERICA': '021',
      'US': '840',
      'WORLD': '001'},
     'statistical_groupings': ['MEDC']},
    'callingcode': 1,
    'currency': {'alternate_symbols': ['US$'],
     'decimal_mark': '.',
     'disambiguate_symbol': 'US$',
     'html_entity': '$',
     'iso_code'

In [26]:
print(f"OSM: {response_coordinates['results'][0]['annotations']['OSM']['url']}")

OSM: https://www.openstreetmap.org/?mlat=29.68313&mlon=-95.21651#map=16/29.68313/-95.21651


In [39]:
print(f"Latitude: {response_coordinates['results'][0]['geometry']['lat']}")
print(f"Longitude: {response_coordinates['results'][0]['geometry']['lng']}")

Latitude: 29.68313
Longitude: -95.216511


In [35]:
def opencagedata_coordinates(address):
    print('----------------------------------------')
    # Forward Geocoding API Endpoint
    url_opencagedata = 'https://api.opencagedata.com/geocode/v1/json?q='
    api_KEY = f"&key={opencagedata_API}"

    # Houston City Hall Coordinates
    HoustonCityHallCoordinates = "29.760376354375307,-95.3702170895345"
    proximity=f"{HoustonCityHallCoordinates}"
    # Bounds for Houston (https://opencagedata.com/bounds-finder)
    bounds='-95.91830,29.35389,-94.80848,30.26282'  
    
    # Format address to look up
    print(f"Address: {address}")
    address_lookup = address.replace(" ", "%20")
    # Format URL to query
    url_coordinates = f"{url_opencagedata}{address_lookup}{api_KEY}&bounds={bounds}&proximity={proximity}"

    
#     print(f"Query URL:")
#     print(url_coordinates)
#     print('')

    #  Perform a request for data
    response_coordinates = requests.get(url_coordinates).json()
    
    
    # Handle empty results
    if len(response_coordinates['results']) == 0:
        valid_result = False
        print("No result found")
        return {"Valid": valid_result}
        
    else:
        valid_result = True
#         print(response_coordinates)
    
        # return coordinates
        latitude = response_coordinates['results'][0]['geometry']['lat']
        longitude = response_coordinates['results'][0]['geometry']['lng']
        
        map_url = f"{response_coordinates['results'][0]['annotations']['OSM']['url']}"
        print(f"Map: {map_url}")

        return {"Valid": valid_result,"latitude": latitude, "longitude": longitude, "map_url": map_url}


### Problematic addresses on nominatin and opentrack

In [34]:
Addresses =['1717 Allendale Bluff Ln, Pearland, TX 77089',
          '15647 Baronial Castle Dr, Humble, TX 77346',
          '2716 Georgina Rose Way, Pearland, TX 77089',
          '4447 Whitney Oak Ln, Houston, TX 77018',
           '2106 Sam Wilson St, Houston, TX 77020']

for address in Addresses:
    opencagedata_coordinates(address)

----------------------------------------
Address: 1717 Allendale Bluff Ln, Pearland, TX 77089
Map: https://www.openstreetmap.org/?mlat=29.68313&mlon=-95.21651#map=16/29.68313/-95.21651
----------------------------------------
Address: 15647 Baronial Castle Dr, Humble, TX 77346
Map: https://www.openstreetmap.org/?mlat=30.19967&mlon=-95.55518#map=16/30.19967/-95.55518
----------------------------------------
Address: 2716 Georgina Rose Way, Pearland, TX 77089
Map: https://www.openstreetmap.org/?mlat=29.71730&mlon=-95.42149#map=16/29.71730/-95.42149
----------------------------------------
Address: 4447 Whitney Oak Ln, Houston, TX 77018
Map: https://www.openstreetmap.org/?mlat=29.83049&mlon=-95.40278#map=16/29.83049/-95.40278
----------------------------------------
Address: 2106 Sam Wilson St, Houston, TX 77020
Map: https://www.openstreetmap.org/?mlat=29.78028&mlon=-95.31416#map=16/29.78028/-95.31416


In [36]:
for index,row in listing_df.iterrows():
    print('-------------------------------------------------')
    # Format address to search for coordinates
    address = listing_df.iloc[index].Address
    opencagedata_coordinates(address)

-------------------------------------------------
----------------------------------------
Address: 2106 Sam Wilson St, Houston, TX 77020
Map: https://www.openstreetmap.org/?mlat=29.78028&mlon=-95.31416#map=16/29.78028/-95.31416
-------------------------------------------------
----------------------------------------
Address: 907 Willowick Bay Dr, Houston, TX 77090
Map: https://www.openstreetmap.org/?mlat=29.75184&mlon=-95.50986#map=16/29.75184/-95.50986
-------------------------------------------------
----------------------------------------
Address: 6602 Preston Trail Dr, Houston, TX 77069
Map: https://www.openstreetmap.org/?mlat=29.98227&mlon=-95.53467#map=17/29.98227/-95.53467
-------------------------------------------------
----------------------------------------
Address: 5835 Duxbury St, Houston, TX 77035
Map: https://www.openstreetmap.org/?mlat=29.64782&mlon=-95.49097#map=16/29.64782/-95.49097
-------------------------------------------------
--------------------------------

Map: https://www.openstreetmap.org/?mlat=29.80810&mlon=-95.49846#map=16/29.80810/-95.49846
-------------------------------------------------
----------------------------------------
Address: 115 Delaney St, Houston, TX 77009
Map: https://www.openstreetmap.org/?mlat=29.81304&mlon=-95.37072#map=17/29.81304/-95.37072
-------------------------------------------------
----------------------------------------
Address: 4008 Black Locust Dr, Houston, TX 77088
Map: https://www.openstreetmap.org/?mlat=29.87146&mlon=-95.46436#map=16/29.87146/-95.46436
-------------------------------------------------
----------------------------------------
Address: 7022 W 43rd St, Houston, TX 77092
Map: https://www.openstreetmap.org/?mlat=29.83283&mlon=-95.47136#map=17/29.83283/-95.47136
-------------------------------------------------
----------------------------------------
Address: 4450 Tidwell Rd Unit K, Houston, TX 77093
Map: https://www.openstreetmap.org/?mlat=29.84871&mlon=-95.33609#map=16/29.84871/-95.3